<a href="https://colab.research.google.com/github/meiruv/IMDB_sentiment_analysis/blob/main/Hugging_Face_Course_On_IMDB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading

In [1]:
reviewsFile = open('reviews.txt','r')
reviews = list(map(lambda x:x[:-1],reviewsFile.readlines()))
reviewsFile.close()

labelsFile = open('labels.txt','r')
labels = list(map(lambda x:x[:-1],labelsFile.readlines()))
labelsFile.close()

print('# of reviews: ', len(reviews))
print('# of labels: ', len(labels))
print(set(labels))

# of reviews:  7829
# of labels:  25000
{'negative', 'positive'}


# First and Easiest Way To Use Huggingface's Models - **Pipeline**

In [29]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(reviews[0])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'NEGATIVE', 'score': 0.9980175495147705}]

In [34]:
classifier(reviews[2])

[{'label': 'NEGATIVE', 'score': 0.9877838492393494}]

In [35]:
classifier(reviews[3])
# How do I handle such errors? ????????????????????????

Token indices sequence length is longer than the specified maximum sequence length for this model (841 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: ignored

# Applying all parts of a huggingFace pipeline
 as described here https://huggingface.co/course/chapter2/2?fw=pt

In [ ]:
!pip install datasets transformers[sentencepiece]


In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [5]:
print(classifier(reviews[0]), 'true label: ', labels[0])


[{'label': 'NEGATIVE', 'score': 0.9980175495147705}] true label:  positive


In [6]:
print(classifier(reviews[1]), 'true label: ', labels[1])


[{'label': 'NEGATIVE', 'score': 0.6425644159317017}] true label:  negative


## Tokenizing

In [7]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
raw_inputs = reviews[:2]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101, 22953,  2213,  4381,  2152,  2003,  1037,  9476,  4038,  1012,
          2009,  2743,  2012,  1996,  2168,  2051,  2004,  2070,  2060,  3454,
          2055,  2082,  2166,  2107,  2004,  5089,  1012,  2026,  2086,  1999,
          1996,  4252,  9518,  2599,  2033,  2000,  2903,  2008, 22953,  2213,
          4381,  2152,  1055, 18312,  2003,  2172,  3553,  2000,  4507,  2084,
          2003,  5089,  1012,  1996, 25740,  2000,  5788, 13732,  1996, 12369,
          3993,  2493,  2040,  2064,  2156,  2157,  2083,  2037, 17203,  5089,
         13433,  8737,  1996,  9004, 10196,  4757,  1997,  1996,  2878,  3663,
          2035, 10825,  2033,  1997,  1996,  2816,  1045,  2354,  1998,  2037,
          2493,  1012,  2043,  1045,  2387,  1996,  2792,  1999,  2029,  1037,
          3076,  8385,  2699,  2000,  6402,  2091,  1996,  2082,  1045,  3202,
          7383,  1012,  1012,  1012,  1012,  1012,  1012,  1012,  1012,  1012,
          2012,  1012,  1012,  1012,  

## Going Through the Model

In [10]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)


torch.Size([2, 182, 768])


The vector output by the Transformer module is usually large. It generally has three dimensions:

Batch size: The number of sequences processed at a time (2 in our example).

Sequence length: The length of the numerical representation of the sequence (16 in our example)

Hidden size: The vector dimension of each model input.

In [20]:
#  doing the same thing only this time using the example from the tutorial
raw_inputs1 = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]
inputs1 = tokenizer(raw_inputs1, padding=True, truncation=True, return_tensors="pt")
print(inputs1)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [21]:
outputs1 = model(**inputs1)
print(outputs1.last_hidden_state.shape)

torch.Size([2, 16, 768])


why do we only have last hidden state in outputs? I thought there should be also attentions and other things...

# Going Through The Model

In [23]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

# How is the above segment different from the following one: ?????????????????

# from transformers import AutoModel
# checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# model = AutoModel.from_pretrained(checkpoint)


In [24]:
print(outputs.logits.shape)


torch.Size([2, 2])


In [25]:
print(outputs.logits)

tensor([[ 3.3620, -2.8595],
        [ 0.4382, -0.1483]], grad_fn=<AddmmBackward>)


In [26]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.9980, 0.0020],
        [0.6426, 0.3574]], grad_fn=<SoftmaxBackward>)


In [27]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

# Models

https://huggingface.co/course/chapter2/3?fw=pt

The AutoModel class and all of its relatives are actually simple wrappers over the wide variety of models available in the library. It’s a clever wrapper as it can automatically guess the appropriate model architecture for your checkpoint, and then instantiates a model with this architecture.

However, if you know the type of model you want to use, you can use the class that defines its architecture directly. Let’s take a look at how this works with a BERT model.

# Different ways to load a model

## 1. from the **default configuration** - initializes it with random values

In [36]:
from transformers import BertConfig, BertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

In [37]:
print(config)


BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## Loading Pre Trained Model

In [38]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Why AutoModel is better than BertModel -

we could replace BertModel with the equivalent AutoModel class. We’ll do this from now on as this **produces checkpoint-agnostic code**; if your code works for one checkpoint, it should work seamlessly with another. This applies even if the architecture is different, as long as the checkpoint was trained for a similar task (for example, a sentiment analysis task).

## Good to know
The weights have been downloaded and cached (so future calls to the from_pretrained method won’t re-download them) in the cache folder, which defaults to ~/.cache/huggingface/transformers. You can customize your cache folder by setting the HF_HOME environment variable.

## Saving a Model

In [39]:
model.save_pretrained("dir_john")


## What we got

1. **config.json** file - contains the attributes necessary to build the model architecture. This file also contains some metadata, such as where the checkpoint originated and what 🤗 Transformers version you were using when you last saved the checkpoint.

2. **pytorch_model.bin** file - known as the state dictionary; it contains all your model’s weights. 

The two files go hand in hand; the configuration is necessary to know your model’s architecture, while the model weights are your model’s parameters.

# Using a Transformer model for inference


In [41]:
sequences = [
  "Hello!",
  "Cool.",
  "Nice!"
]

encoded_sequences = [
  [ 101, 7592,  999,  102],
  [ 101, 4658, 1012,  102],
  [ 101, 3835,  999,  102]
]

import torch

model_inputs = torch.tensor(encoded_sequences)

output = model(model_inputs)


In [42]:
output

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[ 4.4496e-01,  4.8276e-01,  2.7797e-01,  ..., -5.4033e-02,
                                                          3.9394e-01, -9.4770e-02],
                                                        [ 2.4943e-01, -4.4093e-01,  8.1772e-01,  ..., -3.1917e-01,
                                                          2.2992e-01, -4.1172e-02],
                                                        [ 1.3668e-01,  2.2518e-01,  1.4502e-01,  ..., -4.6915e-02,
                                                          2.8224e-01,  7.5566e-02],
                                                        [ 1.1789e+00,  1.6739e-01, -1.8187e-01,  ...,  2.4671e-01,
                                                          1.0441e+00, -6.1963e-03]],
                                               
                                                       [[ 3.6436e-01,  3.2465e-02,  2.0

# Tokenization

 = The task of converting raw text to numbers
 
https://huggingface.co/course/chapter2/4?fw=pt

## Word Based -> Char Based -> Subwords Based

Subword tokenization algorithms rely on the principle that frequently used words should not be split into smaller subwords, but rare words should be decomposed into meaningful subwords.


These subwords end up providing a lot of semantic meaning: for instance, in the example above “tokenization” was split into “token” and “ization”, two tokens that have a semantic meaning while being space-efficient (only two tokens are needed to represent a long word). This allows us to have relatively good coverage with small vocabularies, and close to no unknown tokens.




In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer(reviews[0])


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

{'input_ids': [101, 9304, 4165, 3192, 1344, 1110, 170, 11540, 3789, 119, 1122, 1868, 1120, 1103, 1269, 1159, 1112, 1199, 1168, 2648, 1164, 1278, 1297, 1216, 1112, 4952, 119, 1139, 1201, 1107, 1103, 3679, 9545, 1730, 1143, 1106, 2059, 1115, 9304, 4165, 3192, 1344, 188, 20817, 1110, 1277, 2739, 1106, 3958, 1190, 1110, 4952, 119, 1103, 188, 1665, 4515, 2165, 1106, 5195, 14396, 1103, 14222, 2365, 1651, 1150, 1169, 1267, 1268, 1194, 1147, 18970, 4952, 185, 4165, 1643, 1103, 11109, 12569, 3954, 1104, 1103, 2006, 2820, 1155, 11484, 1143, 1104, 1103, 2126, 178, 1450, 1105, 1147, 1651, 119, 1165, 178, 1486, 1103, 2004, 1107, 1134, 170, 2377, 8038, 1793, 1106, 6790, 1205, 1103, 1278, 178, 2411, 6901, 119, 119, 119, 119, 119, 119, 119, 119, 119, 1120, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 1344, 119, 170, 5263, 1413, 17672, 178, 182, 1303, 1106, 14204, 1141, 1104, 1240, 4952, 119, 2377, 7236, 1106, 9304, 4165, 3192, 1344, 119, 178, 5363, 1115, 1242, 6323, 1104, 1139, 1425, 1341, 1115, 

In [44]:
tokenizer.save_pretrained("dir_john")


('dir_john/tokenizer_config.json',
 'dir_john/special_tokens_map.json',
 'dir_john/vocab.txt',
 'dir_john/added_tokens.json',
 'dir_john/tokenizer.json')

# Encoding

1. split the text into words (or parts of words, punctuation symbols, etc.), usually called tokens. There are multiple rules that can govern that process, which is why we need to instantiate the tokenizer using the name of the model, to make sure we use the same rules that were used when the model was pretrained.

2. convert those tokens into numbers, so we can build a tensor out of them and feed them to the model. To do this, the tokenizer has a vocabulary, which is the part we download when we instantiate it with the from_pretrained method. Again, we need to use the same vocabulary used when the model was pretrained.

In [45]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = reviews[1]
tokens = tokenizer.tokenize(sequence)

print(tokens)

ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

['story', 'of', 'a', 'man', 'who', 'has', 'unnatural', 'feelings', 'for', 'a', 'pig', '.', 'starts', 'out', 'with', 'a', 'opening', 'scene', 'that', 'is', 'a', 'te', '##rri', '##fic', 'example', 'of', 'absurd', 'comedy', '.', 'a', 'formal', 'orchestra', 'audience', 'is', 'turned', 'into', 'an', 'insane', 'violent', 'mob', 'by', 'the', 'crazy', 'chanting', '##s', 'of', 'it', 's', 'singers', '.', 'unfortunately', 'it', 'stays', 'absurd', 'the', 'whole', 'time', 'with', 'no', 'general', 'narrative', 'eventually', 'making', 'it', 'just', 'too', 'off', 'putting', '.', 'even', 'those', 'from', 'the', 'era', 'should', 'be', 'turned', 'off', '.', 'the', 'cry', '##ptic', 'dialogue', 'would', 'make', 'shakes', '##pear', '##e', 'seem', 'easy', 'to', 'a', 'third', 'grade', '##r', '.', 'on', 'a', 'technical', 'level', 'it', 's', 'better', 'than', 'you', 'might', 'think', 'with', 'some', 'good', 'cinema', '##tography', 'by', 'future', 'great', 'v', '##il', '##mos', 'z', '##si', '##g', '##mond', '.',

In [46]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


# Questions Section


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

In [53]:
input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward>)


# ToDo
✏️ Try it out! Convert this batched_ids list into a tensor and pass it through your model. Check that you obtain the same logits as before (but twice)!

# Padding



In [56]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [[200, 200, 200], [200, 200, tokenizer.pad_token_id]]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)
tensor([[ 1.5694, -1.3895],
        [ 1.3373, -1.2163]], grad_fn=<AddmmBackward>)


# There is an issue here-

There’s something wrong with the logits in our batched predictions: the second row should be the same as the logits for the second sentence, but we’ve got completely different values!

This is because the key feature of Transformer models is attention layers that contextualize each token. These will take into account the padding tokens since they attend to all of the tokens of a sequence. To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask.

##**In my words**✈
we got different logits for sequence2 because the transformer set attention on the pad token as well. to avoid this we use **attention mask** which basically tells the transformer to which tokens in the sentence pay attention and which are just paddings...

But how do you set padding automatically on a large corpus? (like the IMDB dataset for example)

##Attention Masks
 are tensors with the exact same shape as the input IDs tensor, filled with 0s and 1s: 1s indicate the corresponding tokens should be attended to, and 0s indicate the corresponding tokens should not be attended to (i.e., they should be ignored by the attention layers of the model).

In [57]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

attention_mask = [
  [1, 1, 1],
  [1, 1, 0]
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)


# Solutions for longer sequences
With Transformer models, there is a limit to the lengths of the sequences we can pass the models. Most models handle sequences of up to 512 or 1024 tokens, and will crash when asked to process longer sequences. There are two solutions to this problem:

Use a model with a longer supported sequence length.
Truncate your sequences.
Models have different supported sequence lengths, and some specialize in handling very long sequences. Longformer is one example, and another is LED. If you’re working on a task that requires very long sequences, we recommend you take a look at those models.

Otherwise, we recommend you truncate your sequences by specifying the max_sequence_length parameter:

Copied
sequence = sequence[:max_sequence_length]

# How to tokenize several sequences



In [60]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = reviews[0:5]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

In [61]:
output

SequenceClassifierOutput([('logits', tensor([[ 3.3620, -2.8595],
                                   [ 0.4382, -0.1483],
                                   [ 2.3844, -2.0083],
                                   [ 2.4363, -2.0455],
                                   [-3.1003,  3.2769]], grad_fn=<AddmmBackward>))])